In [42]:
import os
import warnings
import numpy as np
from skimage import io
import skimage.feature as ft
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from io import BytesIO

url = "http://sipi.usc.edu/database/download.php?vol=textures&img="

def makedata(Nclass, W=40, Nrep=1):
    dataset=[]         #Each row corresponds to a window and contains LBP features of the window
    dataset2=[]        #Each row contains W**2 original pixels (flattened for standard ML tools)
    dataset3=[]        #WxW pixels (good for CNN networks)
    target=[]          #class labels

    # settings for LBP
    METHOD = 'uniform'
    P = 8
    R = 1
    P2 = 16
    R2 = 2

    #Create dataset
    S=512    
    maxpos=S-W
    #part a
    imgList = []
    for ii in range(Nrep):       #Create a random permutation of 13 classes of textures Nrep times
        for i in np.random.permutation(13):

            filename = '1.1.'+str(i+1).zfill(2)
            #filename = '1.1.'+str(ii+1).zfill(2) #get random chuck of each image
            response = requests.get(url + filename)
            img = Image.open(BytesIO(response.content))
            #imgList[img+ii] #load all images into array
            for j in range(Nclass):
                left=np.random.randint(0,maxpos+1)
                top=np.random.randint(0,maxpos+1)

                #win=img[left:left+W,top:top+W]
                imgarray = np.array(img)
                win = imgarray[left:left+W, top:top+W]

                #LBP features
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    feats=ft.local_binary_pattern(win, P, R, METHOD)
                    hist, _ = np.histogram(feats[R:-R][R:-R], normed=True, bins=P + 2, range=(0, P + 2))
                    feats2=ft.local_binary_pattern(win, P2, R2, METHOD)
                    hist2, _ = np.histogram(feats2[R2:-R2][R2:-R2], normed=True, bins=P2 + 2, range=(0, P2 + 2))

                #Append the features of the window to the dataset
                dataset.append(np.concatenate((hist, hist2)))
                dataset2.append(win.flatten())
                dataset3.append(win)
                
                #Window is coming from image i, so save it as the class label
                target.append(i+1)
                
                
    dataset=np.array(dataset)    
    target=np.array(target)    
    return dataset, dataset2, dataset3, target


#Number of windows per texture
nclas=20

for W in np.arange(51,18,-16):   #Try different window sizes: 51, 36, 19
    dataset, dataset2, dataset3, target = makedata(50+nclas,W,10)

    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    
    X_train, X_test, y_train, y_test = train_test_split(dataset,target,train_size=nclas*13, random_state=315, stratify=target)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(dataset2,target,train_size=nclas*13, random_state=315, stratify=target)
    X_train3, X_test3, y_train3, y_test3 = train_test_split(dataset3,target,train_size=nclas*13, random_state=315, stratify=target)

    knn=KNeighborsClassifier(n_neighbors=1, metric='manhattan')
    knn.fit(X_train,y_train)
    preds = knn.predict(X_test)
    knn_acc = np.mean(preds==y_test)

    clf = LogisticRegression(random_state=0).fit(X_train,y_train)    
    preds = clf.predict(X_test)
    lr_acc = np.mean(preds==y_test)

    clf = DecisionTreeClassifier().fit(X_train,y_train)    
    preds = clf.predict(X_test)
    dt_acc = np.mean(preds==y_test)

    print('Using LBP... W=', W, 'knn=', knn_acc, 'lr=', lr_acc, 'dt=', dt_acc)

    knn=KNeighborsClassifier(n_neighbors=1, metric='manhattan')
    knn.fit(X_train2,y_train2)
    preds = knn.predict(X_test2)
    knn_acc = np.mean(preds==y_test2)

    clf = LogisticRegression(random_state=0).fit(X_train2,y_train2)    
    preds = clf.predict(X_test2)
    lr_acc = np.mean(preds==y_test2)

    clf = DecisionTreeClassifier().fit(X_train2,y_train2)    
    preds = clf.predict(X_test2)
    dt_acc = np.mean(preds==y_test2)

    print('Using raw pixels... W=', W, 'knn=', knn_acc, 'lr=', lr_acc, 'dt=', dt_acc)

'''
  A.)
    [img1, true]
    [img2, false]     getimgs
    ....              getclassifiers
    ....
    [img13, false]
'''


Using LBP... W= 51 knn= 0.903054298642534 lr= 0.47907239819004527 dt= 0.748868778280543


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Using raw pixels... W= 51 knn= 0.24423076923076922 lr= 0.17364253393665158 dt= 0.20361990950226244
Using LBP... W= 35 knn= 0.7914027149321267 lr= 0.4468325791855204 dt= 0.6236425339366516


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Using raw pixels... W= 35 knn= 0.251131221719457 lr= 0.15599547511312217 dt= 0.22850678733031674
Using LBP... W= 19 knn= 0.43540723981900453 lr= 0.36493212669683256 dt= 0.34604072398190044
Using raw pixels... W= 19 knn= 0.2694570135746606 lr= 0.1331447963800905 dt= 0.21368778280542985


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


'\n  A.)\n    [img1, true]\n    [img2, false]     getimgs\n    ....              getclassifiers\n    ....\n    [img13, false]\n'